In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from mymodule.func import *
from datetime import date

sns.set(font_scale = 1.2)
np.set_printoptions(suppress=True)
pd.set_option("display.max_rows", 100)

## mESC preprocessing

In [2]:
df = readDF('GSE137764_mESC_Gaussiansmooth_scaled_autosome.mat.gz', drop_labels=None) #2.35% for only repliseq data

GSE137764_mESC_Gaussiansmooth_scaled_autosome.mat.gz with 2.09% missing


In [3]:
# fillNA, chr by chr
df_filled = pd.DataFrame(columns=df.columns, index=df.index, data=np.zeros_like(df.values))

for chr_idx in range(1,20,1):
    temp_cols = getChrCols(df, chr_idx=chr_idx)
    df_filled.loc[:, temp_cols] = fillNAnearest(df.loc[:, temp_cols])
    
if len(getNAcols(df_filled)) == 0:
    print('no nans after filling')

no nans after filling


In [5]:
soft_labels = readDF('MSIdelGeneCoordinatesNEW_bulat.txt', drop_labels=None)
soft_labels = soft_labels.loc[~(soft_labels['chr']=='chrX'), :]
soft_labels = soft_labels.sort_values(by='chr', key=lambda col: [int(x[3:]) for x in col] ).reset_index(drop=True, inplace=False)

# get bins for genes of interest

l = []
for chr_idx in range(1,20,1):
    out = getBinID(soft_labels, chr_idx=chr_idx)
    if out.isna().any().any():
        pass
    else:
        out['bins'] = out.apply(bin_counter, axis=1)
        l.append(out)
df_genes_bins = pd.concat(l)
print(f'{df_genes_bins.shape[0]} genes')

MSIdelGeneCoordinatesNEW_bulat.txt with 0.0% missing
624 genes


In [6]:
# reading in the genic coordinates of interest. insertions
# soft_labels1 = readDF('hotspotInsertionsGeneCoordinatesNEW_bulat.txt', drop_labels=None) <- old list of genes
soft_labels1 = readDF('MSIinsGeneCoordinatesNEW_bulat.txt', drop_labels=None)
soft_labels1 = soft_labels1.loc[~(soft_labels1['chr']=='chrX'), :]
soft_labels1 = soft_labels1.sort_values(by='chr', key=lambda col: [int(x[3:]) for x in col] ).reset_index(drop=True, inplace=False)

# get bins for genes of interest

l1 = []
for chr_idx in range(1,20,1):
    out = getBinID(soft_labels1, chr_idx=chr_idx)
    if out.isna().any().any():
        pass
    else:
        out['bins'] = out.apply(bin_counter, axis=1)
        l1.append(out)
df_genes_bins1 = pd.concat(l1)
print(f'{df_genes_bins1.shape[0]} genes')

MSIinsGeneCoordinatesNEW_bulat.txt with 0.0% missing
84 genes


In [7]:
# gaussian smoothing
df_filled_smoothed = pd.DataFrame(columns=df_filled.columns, index=df_filled.index, data=np.zeros_like(df_filled.values))
df_filled_smoothed.iloc[2:,:] = gausssmoothing(df_filled.iloc[2:,:].values)
df_filled_smoothed.iloc[:2,:] = df_filled.iloc[:2,:]

# scaling to sum=100 for each column
df_filled_smoothed_scaled = pd.DataFrame(columns=df_filled_smoothed.columns, index=df_filled_smoothed.index, data=np.zeros_like(df_filled_smoothed.values))
df_filled_smoothed_scaled.iloc[2:,:] = scalingto100range(df_filled_smoothed.iloc[2:,:].values)
df_filled_smoothed_scaled.iloc[:2,:] = df_filled_smoothed.iloc[:2,:]

if len(getNAcols(df_filled_smoothed_scaled)) == 0:
    print('no nans after smoothing/scaling')

 16/16... rate=0.14 Hz, eta=0:00:00, total=0:01:57, wall=09:36 UTC
no nans after smoothing/scaling


In [8]:
df_filled_smoothed_scaled1 = df_filled_smoothed_scaled # since i dont wanna rename anything in the downstream cells

In [9]:
holder=dict()

for chr_idx in range(1,20):

    df_test = df_filled_smoothed_scaled1.loc[2:, getChrCols(df, chr_idx=chr_idx)].copy() # df for single chr. smoothed & imputed

    s_s50 = df_test.apply(getS50, step=0.25) # s50 profile pd.series
    s_sdiff = df_test.apply(getSdiff, step=0.25) # IQR profiles of s50 pd.series

    l_bins_noisy = filterNoisy(s_sdiff, filter_Z_at=1.65, filter_bins_at=10) # Noisy cols
    l_na = getNAcols(df.loc[2:, getChrCols(df, chr_idx=chr_idx)]) # NA cols
    ar_skipping_bin = np.in1d(s_s50.index, np.unique(np.concatenate([l_na, l_bins_noisy]))) # binary

    # IZ
    prior = list(set(l_na+l_bins_noisy))
    iz_bins = getIZ_short(s_s50.where(ar_skipping_bin==False, other=-1), prominence=0.01, return_plateau=True).tolist()
    iz_bins = [x for x in iz_bins if x not in prior]
    # terminations
    prior = list(set(prior+iz_bins))
    term_bins = getTermination_short(s_s50.where(ar_skipping_bin==False, other=-1), plateau_size=[1,3], prominence=0.05, return_plateau=True).tolist()
    term_bins = [x for x in term_bins if x not in prior]
    # ttr & breaks
    prior = list(set(prior+term_bins))    
    ttr_breaks_bins = list(chain.from_iterable(getTTRsBreaks_short(s_s50, bins_skip=[l_na, l_bins_noisy])))
    ttr_breaks_bins = [x for x in ttr_breaks_bins if x not in prior]
    # ctr
    prior = list(set(prior+ttr_breaks_bins))
    ctr_bins = list(chain.from_iterable(getCTR_short(s_s50)))
    ctr_bins = [x for x in ctr_bins if x not in prior]
    # other
    prior = list(set(prior+ctr_bins))    
    other = [x for x in s_s50.index if x not in prior]
        
    holder[str(chr_idx)]={'s50':s_s50, 'iz':iz_bins, 'termination':term_bins, 'ttr_breaks':ttr_breaks_bins, 'ctr':ctr_bins, 'noisy': list(set(l_na+l_bins_noisy)), 'other':other}


In [10]:
holder_frac={}
for chr_idx, val in holder.items():
    s=len(val["s50"])
    iz = round(100*len(val["iz"])/s, 2)
    termination = round(100*len(val["termination"])/s, 2)
    ttr_breaks = round(100*len(val["ttr_breaks"])/s, 2)
    ctr = round(100*len(val["ctr"])/s, 2)
    other = round(100*len(val["other"])/s, 2)
    noisy = round(100*len(val["noisy"])/s, 2)
    holder_frac[chr_idx] = {'iz':iz, 'termination':termination, 'ttr_breaks':ttr_breaks, 'ctr':ctr, 'other':other, 'noisy':noisy}

In [11]:
izs = [x['iz'] for x in holder_frac.values()]
terminations = [x['termination'] for x in holder_frac.values()]
ttr_breaks = [x['ttr_breaks'] for x in holder_frac.values()]
ctrs = [x['ctr'] for x in holder_frac.values()]
others = [x['other'] for x in holder_frac.values()]
noisy = [x['noisy'] for x in holder_frac.values()]

print([(label, round(np.mean(x),2) ) for x,label in zip([izs, terminations, ttr_breaks, ctrs, others, noisy], ['izs', 'terminations', 'ttr_breaks', 'ctrs', 'others', 'noisy']) ])

[('izs', 9.6), ('terminations', 4.46), ('ttr_breaks', 60.25), ('ctrs', 7.25), ('others', 15.0), ('noisy', 3.44)]


### genome vs exome

In [12]:
l_izs = list(chain.from_iterable([x["iz"] for x in holder.values()]))
l_terminations = list(chain.from_iterable([x["termination"] for x in holder.values()]))
l_ttr_breaks = list(chain.from_iterable([x["ttr_breaks"] for x in holder.values()]))
l_ctrs = list(chain.from_iterable([x["ctr"] for x in holder.values()]))
l_noisy = list(chain.from_iterable([x["noisy"] for x in holder.values()]))
l_other = list(chain.from_iterable([x["other"] for x in holder.values()]))

In [13]:
exome=pd.read_csv('exome_100bp_padding.bed', sep='\t', header=None)
exome.columns = ['chr','start','end']

In [14]:
exome = exome.loc[~(exome['chr']=='chrX'), :]
exome = exome.loc[~(exome['chr']=='chrY'), :]
exome = exome.loc[~(exome['chr']=='chrM'), :]
exome = exome.sort_values(by='chr', key=lambda col: [int(x[3:]) for x in col] ).reset_index(drop=True, inplace=False)


In [15]:
l2 = []
for chr_idx in range(1,20,1):
    out = getBinID(exome, chr_idx=chr_idx, is_gene=False)
    if out.isna().any().any():
        pass
    else:
        out['bins'] = out.apply(bin_counter, axis=1)
        l2.append(out)
df_genes_bins2 = pd.concat(l2)

In [16]:
exome_bins={}
for chr_idx in range(1,20):
    c = 'chr'+str(chr_idx)
    exome_bins[c]=list(set(chain.from_iterable(df_genes_bins2[df_genes_bins2['chr']==c].bins.values.tolist())))

In [17]:
l_all_exome=[]
idx=[]
for c, bins in exome_bins.items():
    str_bins=[c+'.'+str(subbin) for subbin in bins]   
    idx.append(c)
    verdict=[]
    for b in str_bins:
        if b in l_izs:
            verdict.append('IZ')
        elif b in l_terminations:
            verdict.append('termination')
        elif b in l_ttr_breaks:
            verdict.append('TTR or breakage')
        elif b in l_ctrs:
            verdict.append('CTR')
        elif b in l_noisy:
            verdict.append('noisy')
        else:
            verdict.append('other')
    l_all_exome.append(pd.DataFrame.from_dict(Counter(verdict), orient='index').T)
df_exome=pd.concat(l_all_exome)
df_exome.fillna(0, inplace=True)
df_exome.index=idx
df_exome.loc[:,'noisy'] = df_exome.loc[:,'noisy'].astype(int)

In [18]:
df_exome.to_csv('exome_counts_per_categ_mESC.csv', sep='\t')

### deletions

In [19]:
izs = list(chain.from_iterable([x["iz"] for x in holder.values()]))
terminations = list(chain.from_iterable([x["termination"] for x in holder.values()]))
ttr_breaks = list(chain.from_iterable([x["ttr_breaks"] for x in holder.values()]))
ctrs = list(chain.from_iterable([x["ctr"] for x in holder.values()]))
noisy = list(chain.from_iterable([x["noisy"] for x in holder.values()]))
other = list(chain.from_iterable([x["other"] for x in holder.values()]))

In [20]:
l_all=[]
for _,x in df_genes_bins.iterrows():
    
    bins=[x['chr']+'.'+str(subbin) for subbin in x['bins']]   
    verdict=[]
    for b in bins:
        if b in l_izs:
            verdict.append('IZ')
        elif b in l_terminations:
            verdict.append('termination')
        elif b in l_ttr_breaks:
            verdict.append('TTR or breakage')
        elif b in l_ctrs:
            verdict.append('CTR')
        elif b in l_noisy:
            verdict.append('noisy')
        else:
            verdict.append('other')
    l_all.append([x['genename'],
                  round(df_filled_smoothed_scaled1.loc[2:, bins].apply(getS50, step=0.1).mean(), 2),
                  #soft_labels[soft_labels['genename']==x['genename']]['replication_feature'].values[0],
                  Counter(verdict).most_common()[0][0],Counter(verdict).most_common()[0][1],
                  Counter(verdict).most_common()[1:]]
                  )

In [21]:
pd.DataFrame(l_all, columns=['genes','s50','calc_most_common_label','most_common_label_count','other_labels&counts'])\
.to_csv(f'deletions_all_{date.today().strftime("%d%m%y")}_mESC.csv', sep='\t', index=None)

### insertions

In [22]:
l_all1=[]
for _,x in df_genes_bins1.iterrows():
    bins=[x['chr']+'.'+str(subbin) for subbin in x['bins']]   
    verdict=[]
    for b in bins:
        if b in l_izs:
            verdict.append('IZ')
        elif b in l_terminations:
            verdict.append('termination')
        elif b in l_ttr_breaks:
            verdict.append('TTR or breakage')
        elif b in l_ctrs:
            verdict.append('CTR')
        elif b in l_noisy:
            verdict.append('noisy')
        else:
            verdict.append('other')
    l_all1.append([x['genename'],
                   round(df_filled_smoothed_scaled1.loc[2:, bins].apply(getS50, step=0.1).mean(), 2),
                  Counter(verdict).most_common()[0][0],Counter(verdict).most_common()[0][1],
                  Counter(verdict).most_common()[1:]]
                  )

In [23]:
pd.DataFrame(l_all1, columns=['genes','s50','calc_most_common_label','most_common_label_count','other_labels&counts'])\
.to_csv(f'insertions_all_{date.today().strftime("%d%m%y")}_mESC.csv', sep='\t', index=None)